In [2]:
import os
import datetime
import pandas as pd
from client import client
    

In [3]:
def try_create_directory(path):
    try:
        os.mkdir(os.path.abspath("") + '/' + path)
    except OSError:
        print ("Creation of the directory failed")
    else:
        print ("Directory created successfully")

try_create_directory(f'data')
try_create_directory(f'data/by-date')
try_create_directory(f'data/by-date-range')


Creation of the directory failed
Creation of the directory failed
Creation of the directory failed


-------------------------------------------------------
Countries

In [4]:
countries = client.get_countries()

data = {'Name': [country['name'] for country in countries]}
df = pd.DataFrame(data, columns=['Name'])
df.to_csv ('data/countries.csv', index = False, header=True)

---------------------------------
Country's today statisctics

In [5]:
def generate_and_save_country_last_stats(country, save=True):
    country_data = client.get_country_detail(country)
    lastUpdate = country_data[0]['lastUpdate']
    date = datetime.datetime.fromtimestamp(lastUpdate/1000.0)

    data = {
        'provinceState': [country_data[i]['provinceState'] for i in range(len(country_data))],
        'Confirmed': [country_data[i]['confirmed'] for i in range(len(country_data))],
        'Recovered': [country_data[i]['recovered'] for i in range(len(country_data))],
        'Deaths': [country_data[i]['deaths'] for i in range(len(country_data))],
        'Active': [country_data[i]['active'] for i in range(len(country_data))],
        'date': [date] * (len(country_data))
    }
    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date.day}-{date.month}-{date.year}')
        df.to_csv (f'data/by-date/{date.day}-{date.month}-{date.year}/{country}.csv', index = False, header=True)

    return data

In [21]:
country = "China"
_ = generate_and_save_country_last_stats(country)

Directory created successfully


_____
Statistics of specific country for given date

In [6]:
def generate_and_save_country_stats_by_date(country, day, month, year, save=True):
    country_data = client.get_country_stats_by_date(country, day, month, year)
    lastUpdate = country_data[0]['lastUpdate']
    date = datetime.datetime.strptime(lastUpdate, '%Y-%m-%d %H:%M:%S')

    data = {
        'provinceState': [country_data[i]['provinceState'] for i in range(len(country_data))],
        'Confirmed': [country_data[i]['confirmed'] for i in range(len(country_data))],
        'Recovered': [country_data[i]['recovered'] for i in range(len(country_data))],
        'Deaths': [country_data[i]['deaths'] for i in range(len(country_data))],
        'Active': [country_data[i]['active'] for i in range(len(country_data))],
        'date': [date] * len(country_data)
    }

    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date.day}-{date.month}-{date.year}')
        df.to_csv (f'data/by-date/{date.day}-{date.month}-{date.year}/{country}.csv', index = False, header=True)
    return data

In [7]:
country = "Kazakhstan"
year = 2021
month = 1
day = 25
_ = generate_and_save_country_stats_by_date(country, day, month, year)

Creation of the directory failed


___________
Statisctics of world situation for given date

In [8]:

def generate_and_save_world_stats_by_date(day, month, year, save=True):
    world_data = client.get_world_stats_by_date(day, month, year)
    lastUpdate = world_data[0]['lastUpdate']
    date = datetime.datetime.strptime(lastUpdate, '%Y-%m-%d %H:%M:%S')

    data = {
        'country': [world_data[i]['countryRegion'] for i in range(len(world_data))],
        'provinceState': [world_data[i]['provinceState'] for i in range(len(world_data))],
        'Confirmed': [world_data[i]['confirmed'] for i in range(len(world_data))],
        'Recovered': [world_data[i]['recovered'] for i in range(len(world_data))],
        'Deaths': [world_data[i]['deaths'] for i in range(len(world_data))],
        'Active': [world_data[i]['active'] for i in range(len(world_data))],
        'date': [date] * len(world_data)
    }

    if save:
        df = pd.DataFrame(data, columns=['country', 'provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date/{date.day}-{date.month}-{date.year}')
        try_create_directory(f'data/by-date/{date.day}-{date.month}-{date.year}/world')
        df.to_csv (f'data/by-date/{date.day}-{date.month}-{date.year}/world/World.csv', index = False, header=True)

    return data


In [26]:
year = 2021
month = 1
day = 25
_ = generate_and_save_world_stats_by_date(day, month, year)

Creation of the directory failed
Creation of the directory failed


_____
Statistics of country for specific period of time

In [9]:
def generate_and_save_contry_stats_in_date_range(country, date1, date2, save=True):
    if len(date1) != 3 or len(date2) != 3:
        raise Exception("Incorrect format of date.")

    start_date = datetime.date(*reversed(date1))
    end_date = datetime.date(*reversed(date2))
    day_count = (end_date - start_date).days + 1

    if day_count <= 0:
        raise Exception("Start date must be before end date.")

    data = {
        'provinceState': [],
        'Confirmed': [],
        'Recovered': [],
        'Deaths': [],
        'Active': [],
        'date': []
    }

    for date in (start_date + datetime.timedelta(n) for n in range(day_count)):
        try:
            date_data = generate_and_save_country_stats_by_date(country, date.day, date.month, date.year, save=False)
            for key in data.keys():
                data[key] += date_data[key]

        except Exception as e:
            print(e)

    if save:
        df = pd.DataFrame(data, columns=['provinceState', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'date'])
        try_create_directory(f'data/by-date-range')
        try_create_directory(f'data/by-date-range/{start_date.day}-{start_date.month}-{start_date.year}to{end_date.day}-{end_date.month}-{end_date.year}')
        df.to_csv (f'data/by-date-range/{start_date.day}-{start_date.month}-{start_date.year}to{end_date.day}-{end_date.month}-{end_date.year}/{country}.csv', index = False, header=True)
    return data


In [10]:
country = "Kazakhstan"
date1 = (1,6,2020)
date2 = (4,6,2020)

_ = generate_and_save_contry_stats_in_date_range(country, date1, date2)

Creation of the directory failed
Creation of the directory failed
